Initialize the embedding function and the ChromaDB client

In [1]:
from chromadb.utils import embedding_functions
import chromadb

huggingface_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

chroma_client = chromadb.Client()
# client = chromadb.PersistentClient(path="/path/to/save/to")
collection = chroma_client.create_collection(name="my_collection", embedding_function=huggingface_ef)


/Users/lucacordioli/Documents/Lavori/polimi/TESI/visionHelperSrv/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/lucacordioli/Documents/Lavori/polimi/TESI/visionHelperSrv/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/lucacordioli/Documents/Lavori/polimi/TESI/visionHelperSrv/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Take data from a pdf file

In [2]:
import fitz
doc = fitz.open("/Users/lucacordioli/Documents/Lavori/polimi/TESI/visionHelperSrv/data/test.pdf")

pdf_texts = ""

for page in doc: # iterate the document pages
    pdf_texts += page.get_text()
    

Add data to chorma db

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=600, chunk_overlap=100)

for i, page in enumerate(doc):
    page_text = page.get_text()
    doc_splits = text_splitter.split_text(page_text)
    collection.add(
        documents=doc_splits,
        metadatas=[{'page': i+1} for _ in doc_splits],
        ids=[f"id_{i}_{j}" for j in range(len(doc_splits))]
    )

Take data from web page

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
urls = [
    "https://www.apple.com/newsroom/2024/05/apple-reports-second-quarter-results/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

Add data to chorma db

In [ ]:
doc_splits = text_splitter.split_documents(docs_list)

collection.add(
    documents=[doc.page_content for doc in doc_splits],
    metadatas=[doc.metadata for doc in doc_splits],
    ids=[f"id_{doc}" for doc in range(len(doc_splits))]
)

Query to ChromaDB

In [ ]:
question = "What is Luca educational path?"
system_message = ""

results = collection.query(
    query_texts=[question],
    n_results=3,
    include=["documents"]
)

context = ""
for doc in results["documents"]:
    context += "\n\n".join(doc)
    
print(context)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model="llama3:8b", temperature=0, max_tokens=2048,
                 stop=["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>", "<|reserved_special_token"])

prompt = PromptTemplate(
    template="""<|begin_of_text|>
      <|start_header_id|>system<|end_header_id|>You are a question-answer task assistant. Use the following portion of the context to answer the question. If you do not know the answer, simply answer that you do not know the answer. Use a maximum of 3 sentences to answer concisely.<|eot_id|>
      <|start_header_id|>user<|end_header_id|>
      Question: {question}
      Context: {context}
      Answer: <|eot_id|>
      <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["system_message", "question", "context"],
)

chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"question": question, "system_message": system_message, "context": context}):
    print(chunk, end="", flush=True)

Load json file

In [7]:
import json

index = 3

with open('/Users/lucacordioli/Documents/Lavori/polimi/TESI/visionHelperSrv/data/data.json', 'r') as file:
    data = json.load(file)
    
    obj = data['medias'][index]
    
    item_id = obj['id']
    name = obj['name']
    caption = obj['caption']
    guid = obj['guid']
    media_type = obj['mediaType']['value']

    query_text = f"ID: {item_id}, Name: {name}, Caption: {caption}, GUID: {guid}, MediaType: {media_type}"
    
    print(query_text)


ID: 13483, Name: Hand, Caption: Hand, GUID: 7b6d3c28-5650-49aa-9c6a-aeec1cf5524a, MediaType: 3D Object


Query to ChromaDB

In [8]:
results = collection.query(
    query_texts=[query_text],
    n_results=3,
    include=["documents", "metadatas"]
)

print("Page: " + str(results["metadatas"][0][0]["page"]))
print("Document: " + results["documents"][0][0])

Page: 2
Document: The Finger component is designed to interact with other parts of the assembly, probably 
functioning as a lever or actuator. Its precise engineering allows for ﬁne movements, 
crucial in applications requiring dexterity and accuracy. The Finger is made from 
lightweight yet strong aluminum alloy, balancing strength and maneuverability. 
 
### 3. ForeArm 
- **ID**: 13482 
- **Caption**: ForeArm 
- **File**: ForeArm.glb 
- **Size**: 2,215,632 bytes 
- **Media Type**: 3D Object 
 
#### Description: 
The ForeArm acts as a connector, linking the Finger to other parts of the assembly. Its 
elongated structure enables extended reach and enhanced ﬂexibility. The ForeArm is 
constructed from reinforced composite materials, ensuring it can withstand high stress 
and strain without compromising performance. 
 
### 4. Hand 
- **ID**: 13483 
- **Caption**: Hand 
- **File**: Hand.glb 
- **Size**: 1,185,264 bytes 
- **Media Type**: 3D Object 
 
#### Description: 
The Hand component 